<a href="https://colab.research.google.com/github/konsin1988/machine_learning_in_python/blob/master/LightGBM_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
melb = pd.read_csv('/content/drive/MyDrive/datasets/melbourne_housing_FULL.csv')
melb.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [ ]:
melb.isna().sum()

,0
Suburb,0
Address,0
Rooms,0
Type,0
Price,7610
Method,0
SellerG,0
Date,0
Distance,1
Postcode,1


In [ ]:
melb.iloc[:, :5].head()

,Suburb,Address,Rooms,Type,Price
0,Abbotsford,68 Studley St,2,h,NaN
1,Abbotsford,85 Turner St,2,h,1480000.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0
3,Abbotsford,18/659 Victoria St,3,u,NaN
4,Abbotsford,5 Charles St,3,h,1465000.0


In [ ]:
melb.iloc[:, 5:10].head()

,Method,SellerG,Date,Distance,Postcode
0,SS,Jellis,3/09/2016,2.5,3067.0
1,S,Biggin,3/12/2016,2.5,3067.0
2,S,Biggin,4/02/2016,2.5,3067.0
3,VB,Rounds,4/02/2016,2.5,3067.0
4,SP,Biggin,4/03/2017,2.5,3067.0


In [ ]:
melb.iloc[:, 10:15].head()

,Bedroom2,Bathroom,Car,Landsize,BuildingArea
0,2.0,1.0,1.0,126.0,NaN
1,2.0,1.0,1.0,202.0,NaN
2,2.0,1.0,0.0,156.0,79.0
3,3.0,2.0,1.0,0.0,NaN
4,3.0,2.0,0.0,134.0,150.0


In [ ]:
melb.iloc[:, 15:].head()

,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [ ]:
melb = (
    melb
    .drop(columns = ['Address',
                     'Method',
                     'SellerG',
                     'Date',
                     'CouncilArea',
                     'Lattitude',
                     'Longtitude',
                     'Propertycount'])
)
melb.head()

,Suburb,Rooms,Type,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Regionname
0,Abbotsford,2,h,NaN,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,NaN,Northern Metropolitan
1,Abbotsford,2,h,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Northern Metropolitan
2,Abbotsford,2,h,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Northern Metropolitan
3,Abbotsford,3,u,NaN,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,NaN,Northern Metropolitan
4,Abbotsford,3,h,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Northern Metropolitan


In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Suburb        34857 non-null  object 
 1   Rooms         34857 non-null  int64  
 2   Type          34857 non-null  object 
 3   Price         27247 non-null  float64
 4   Distance      34856 non-null  float64
 5   Postcode      34856 non-null  float64
 6   Bedroom2      26640 non-null  float64
 7   Bathroom      26631 non-null  float64
 8   Car           26129 non-null  float64
 9   Landsize      23047 non-null  float64
 10  BuildingArea  13742 non-null  float64
 11  YearBuilt     15551 non-null  float64
 12  Regionname    34854 non-null  object 
dtypes: float64(9), int64(1), object(3)
memory usage: 3.5+ MB


In [ ]:
melb.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Price,7610
Distance,1
Postcode,1
Bedroom2,8217
Bathroom,8226
Car,8728
Landsize,11810


In [ ]:
melb['YearBuilt'].value_counts(dropna=False)

,count
YearBuilt,
NaN,19306
1970.0,1490
1960.0,1260
1950.0,1089
1980.0,726
...,...
1876.0,1
2106.0,1
1897.0,1


In [ ]:
melb = melb.drop(columns = ['YearBuilt'])
melb.head()

,Suburb,Rooms,Type,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,Regionname
0,Abbotsford,2,h,NaN,2.5,3067.0,2.0,1.0,1.0,126.0,NaN,Northern Metropolitan
1,Abbotsford,2,h,1480000.0,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,Northern Metropolitan
2,Abbotsford,2,h,1035000.0,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,Northern Metropolitan
3,Abbotsford,3,u,NaN,2.5,3067.0,3.0,2.0,1.0,0.0,NaN,Northern Metropolitan
4,Abbotsford,3,h,1465000.0,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,Northern Metropolitan


In [ ]:
melb.count()

,0
Suburb,34857
Rooms,34857
Type,34857
Price,27247
Distance,34856
Postcode,34856
Bedroom2,26640
Bathroom,26631
Car,26129
Landsize,23047


In [ ]:
melb.dropna(axis = 0, how = 'any', subset = ['Distance', 'Postcode', 'Price']).count()

,0
Suburb,27246
Rooms,27246
Type,27246
Price,27246
Distance,27246
Postcode,27246
Bedroom2,20806
Bathroom,20800
Car,20423
Landsize,17982


In [ ]:
melb = melb.dropna(axis = 0, how = 'any', subset = ['Distance', 'Postcode', 'Price'])
melb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27246 entries, 1 to 34856
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Suburb        27246 non-null  object 
 1   Rooms         27246 non-null  int64  
 2   Type          27246 non-null  object 
 3   Price         27246 non-null  float64
 4   Distance      27246 non-null  float64
 5   Postcode      27246 non-null  float64
 6   Bedroom2      20806 non-null  float64
 7   Bathroom      20800 non-null  float64
 8   Car           20423 non-null  float64
 9   Landsize      17982 non-null  float64
 10  BuildingArea  10656 non-null  float64
 11  Regionname    27244 non-null  object 
dtypes: float64(8), int64(1), object(3)
memory usage: 2.7+ MB


In [ ]:
melb['BuildingArea'].value_counts()

,count
BuildingArea,
120.00,152
110.00,128
115.00,126
100.00,123
130.00,121
...,...
167.13,1
443.00,1
104.63,1


In [ ]:
melb = melb.drop(columns = ['BuildingArea'])

In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27246 entries, 1 to 34856
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Suburb      27246 non-null  object 
 1   Rooms       27246 non-null  int64  
 2   Type        27246 non-null  object 
 3   Price       27246 non-null  float64
 4   Distance    27246 non-null  float64
 5   Postcode    27246 non-null  float64
 6   Bedroom2    20806 non-null  float64
 7   Bathroom    20800 non-null  float64
 8   Car         20423 non-null  float64
 9   Landsize    17982 non-null  float64
 10  Regionname  27244 non-null  object 
dtypes: float64(7), int64(1), object(3)
memory usage: 2.5+ MB


In [ ]:
fig = (
    px.imshow(melb.select_dtypes('number').corr(), text_auto = True)

)
fig.show()

In [ ]:
melb = melb.drop(columns = ['Landsize'])

In [ ]:
melb.select_dtypes(exclude='number')

,Suburb,Type,Regionname
1,Abbotsford,h,Northern Metropolitan
2,Abbotsford,h,Northern Metropolitan
4,Abbotsford,h,Northern Metropolitan
5,Abbotsford,h,Northern Metropolitan
6,Abbotsford,h,Northern Metropolitan
...,...,...,...
34852,Yarraville,h,Western Metropolitan
34853,Yarraville,h,Western Metropolitan
34854,Yarraville,t,Western Metropolitan
34855,Yarraville,h,Western Metropolitan


In [ ]:
melb['Suburb'].value_counts().count()

344

In [ ]:
melb_sub = (
    melb
    .assign(Suburb = lambda x:
            x['Suburb'].apply(lambda y:
                          y if x['Suburb'].value_counts()[y] > 200 else 'others'))
)

In [ ]:
melb_sub['Suburb'].value_counts()

,count
Suburb,
others,16089
Reservoir,727
Bentleigh East,493
Richmond,439
Preston,415
Brunswick,387
Essendon,361
Northcote,345
Glenroy,342


In [ ]:
melb = melb_sub.drop(columns = ['Postcode'])

In [ ]:
fig = (
    px.imshow(melb.select_dtypes('number').corr(), text_auto = True)

)
fig.show()

In [ ]:
fig = (
    px.box(melb, y  = 'Rooms')
)
fig.show()

In [ ]:
melb.query('Rooms > 7')

,Suburb,Rooms,Type,Price,Distance,Bedroom2,Bathroom,Car,Regionname
616,others,8,h,2950000.0,11.0,9.0,7.0,4.0,Southern Metropolitan
2613,others,10,h,1960000.0,11.7,NaN,NaN,NaN,Southern Metropolitan
3288,Coburg,8,u,2250000.0,7.8,4.0,2.0,4.0,Northern Metropolitan
5409,others,8,h,770000.0,10.5,8.0,4.0,4.0,Eastern Metropolitan
5985,Kew,8,h,3130000.0,5.6,NaN,NaN,NaN,Southern Metropolitan
7134,Newport,8,h,881000.0,8.4,6.0,2.0,4.0,Western Metropolitan
7535,others,9,h,1380000.0,14.7,NaN,NaN,NaN,South-Eastern Metropolitan
7590,others,8,u,3210000.0,2.6,NaN,NaN,NaN,Northern Metropolitan
7598,others,10,h,2800000.0,2.6,NaN,NaN,NaN,Northern Metropolitan
10157,others,8,h,1150000.0,13.8,6.0,4.0,3.0,Eastern Metropolitan


In [ ]:
melb = melb.drop(axis = 0, index = melb.query('Rooms > 7').index)

In [ ]:
melb.isna().sum()

,0
Suburb,0
Rooms,0
Type,0
Price,0
Distance,0
Bedroom2,6433
Bathroom,6439
Car,6815
Regionname,2


In [ ]:
melb[['Bathroom', 'Bedroom2', 'Car']].isna().all(axis = 1).sum()

KeyError: "['Bedroom2'] not in index"

In [ ]:
melb = melb.drop(columns = ['Bedroom2'])

In [ ]:
melb[['Bathroom', 'Car']].isna().all(axis = 1).sum()

6439

In [ ]:
melb.count()

,0
Suburb,27223
Rooms,27223
Type,27223
Price,27223
Distance,27223
Bathroom,20784
Car,20408
Regionname,27221


In [ ]:
melb = melb.dropna(axis = 0, how = 'any')

In [ ]:
melb.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Regionname
1,others,2,h,1480000.0,2.5,1.0,1.0,Northern Metropolitan
2,others,2,h,1035000.0,2.5,1.0,0.0,Northern Metropolitan
4,others,3,h,1465000.0,2.5,2.0,0.0,Northern Metropolitan
5,others,3,h,850000.0,2.5,2.0,1.0,Northern Metropolitan
6,others,4,h,1600000.0,2.5,1.0,2.0,Northern Metropolitan


In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20408 entries, 1 to 34856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Suburb      20408 non-null  object 
 1   Rooms       20408 non-null  int64  
 2   Type        20408 non-null  object 
 3   Price       20408 non-null  float64
 4   Distance    20408 non-null  float64
 5   Bathroom    20408 non-null  float64
 6   Car         20408 non-null  float64
 7   Regionname  20408 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ MB


In [ ]:
melb['Type'].value_counts()

,count
Type,
h,15350
u,3481
t,1577


In [ ]:
melb['Regionname'].value_counts()

,count
Regionname,
Southern Metropolitan,6215
Northern Metropolitan,5796
Western Metropolitan,4520
Eastern Metropolitan,2503
South-Eastern Metropolitan,1017
Northern Victoria,138
Eastern Victoria,137
Western Victoria,82


In [ ]:
fig = (
    px.box(melb, y = 'Distance')
)
fig.show()

In [ ]:
melb.query('Distance > 30')

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Regionname
14263,others,4,h,627500.0,35.2,2.0,2.0,Eastern Victoria
14326,others,5,h,572000.0,34.7,2.0,1.0,South-Eastern Metropolitan
14384,others,3,h,640000.0,38.0,2.0,1.0,South-Eastern Metropolitan
14386,others,3,h,611000.0,36.9,2.0,1.0,Eastern Victoria
14388,others,3,h,755000.0,38.0,1.0,2.0,South-Eastern Metropolitan
...,...,...,...,...,...,...,...,...
34751,others,4,h,500000.0,31.7,2.0,2.0,Western Metropolitan
34752,others,3,h,556000.0,31.7,2.0,2.0,Western Metropolitan
34753,others,3,h,575000.0,31.7,2.0,2.0,Western Metropolitan
34754,others,4,h,791000.0,31.7,2.0,2.0,Western Metropolitan


In [ ]:
fig = (
    px.box(melb, y = 'Bathroom')
)
fig.show()

In [ ]:
melb.query('Bathroom > 3')

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Regionname
202,others,5,h,2840000.0,6.4,4.0,4.0,Northern Metropolitan
642,others,5,h,3510000.0,11.0,5.0,2.0,Southern Metropolitan
650,others,5,h,2752000.0,11.0,4.0,2.0,Southern Metropolitan
756,others,4,h,1200000.0,10.5,5.0,4.0,Western Metropolitan
856,Balwyn,5,h,3250000.0,9.7,5.0,2.0,Southern Metropolitan
...,...,...,...,...,...,...,...,...
34078,Camberwell,4,h,2700000.0,7.7,4.0,4.0,Southern Metropolitan
34427,others,4,h,2900000.0,5.9,4.0,3.0,Southern Metropolitan
34494,others,6,h,5575000.0,3.0,4.0,4.0,Southern Metropolitan
34682,others,5,h,1055000.0,23.5,4.0,2.0,South-Eastern Metropolitan


In [ ]:
melb = melb.drop(axis = 0, index = melb.query('Bathroom > 3').index)

In [ ]:
fig = (
    px.box(melb, y = 'Bathroom')
)
fig.show()

In [ ]:
fig = (
    px.box(melb, y = 'Price')
)
fig.show()

In [ ]:
melb.query('Price > 6000000')

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Regionname
10451,others,3,h,6250000.0,4.6,3.0,2.0,Southern Metropolitan
15696,Hawthorn,4,h,7650000.0,5.3,2.0,4.0,Southern Metropolitan
19583,others,3,h,9000000.0,18.8,1.0,1.0,South-Eastern Metropolitan
20684,others,5,h,6400000.0,3.0,2.0,1.0,Southern Metropolitan
20924,others,4,h,6800000.0,6.3,2.0,2.0,Southern Metropolitan
24556,others,4,h,6370000.0,3.0,3.0,3.0,Southern Metropolitan
25504,others,4,h,6050000.0,6.3,2.0,2.0,Southern Metropolitan
25635,Brighton,4,h,11200000.0,10.5,3.0,2.0,Southern Metropolitan
26147,Malvern East,5,h,6125000.0,8.4,3.0,2.0,Southern Metropolitan
30551,others,5,h,6600000.0,5.9,3.0,2.0,Southern Metropolitan


In [ ]:
melb.query('Suburb == "Hawthorn"')

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Regionname
5179,Hawthorn,4,h,1650000.0,4.6,2.0,2.0,Southern Metropolitan
5180,Hawthorn,3,h,5100000.0,4.6,2.0,3.0,Southern Metropolitan
5181,Hawthorn,4,h,2670000.0,4.6,2.0,1.0,Southern Metropolitan
5184,Hawthorn,3,h,1480000.0,4.6,1.0,0.0,Southern Metropolitan
5185,Hawthorn,1,u,348000.0,4.6,1.0,1.0,Southern Metropolitan
...,...,...,...,...,...,...,...,...
33413,Hawthorn,5,h,3120000.0,5.3,2.0,3.0,Southern Metropolitan
33414,Hawthorn,2,h,1265000.0,5.3,1.0,2.0,Southern Metropolitan
34326,Hawthorn,3,h,1362500.0,5.3,1.0,1.0,Southern Metropolitan
34329,Hawthorn,5,h,4630000.0,5.3,3.0,2.0,Southern Metropolitan


In [ ]:
fig = (
    px.box(melb.query('Suburb == "Brighton"'),
    y = 'Price')
)
fig.show()

In [ ]:
melb = melb.drop(axis = 0, index = melb.query('Price > 6000000').index)

In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20170 entries, 1 to 34856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Suburb      20170 non-null  object 
 1   Rooms       20170 non-null  int64  
 2   Type        20170 non-null  object 
 3   Price       20170 non-null  float64
 4   Distance    20170 non-null  float64
 5   Bathroom    20170 non-null  float64
 6   Car         20170 non-null  float64
 7   Regionname  20170 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.4+ MB


In [9]:
melb[melb.select_dtypes(exclude = 'number').columns] = melb.select_dtypes(exclude = 'number').astype('category')

NameError: name 'melb' is not defined

In [ ]:
melb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20170 entries, 1 to 34856
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Suburb      20170 non-null  category
 1   Rooms       20170 non-null  int64   
 2   Type        20170 non-null  category
 3   Price       20170 non-null  float64 
 4   Distance    20170 non-null  float64 
 5   Bathroom    20170 non-null  float64 
 6   Car         20170 non-null  float64 
 7   Regionname  20170 non-null  category
dtypes: category(3), float64(4), int64(1)
memory usage: 1006.4 KB


In [ ]:
melb.to_csv('/content/drive/MyDrive/datasets/cleaned_data/melb_cleaned.csv', index = False)

In [5]:
melb_cleaned = pd.read_csv('/content/drive/MyDrive/datasets/cleaned_data/melb_cleaned.csv', )
melb_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20170 entries, 0 to 20169
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Suburb      20170 non-null  object 
 1   Rooms       20170 non-null  int64  
 2   Type        20170 non-null  object 
 3   Price       20170 non-null  float64
 4   Distance    20170 non-null  float64
 5   Bathroom    20170 non-null  float64
 6   Car         20170 non-null  float64
 7   Regionname  20170 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.2+ MB


In [13]:
melb_cleaned[melb_cleaned.select_dtypes(exclude = 'number').columns] = melb_cleaned.select_dtypes(exclude = 'number').astype('category')
melb_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20170 entries, 0 to 20169
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Suburb      20170 non-null  category
 1   Rooms       20170 non-null  int64   
 2   Type        20170 non-null  category
 3   Price       20170 non-null  float64 
 4   Distance    20170 non-null  float64 
 5   Bathroom    20170 non-null  float64 
 6   Car         20170 non-null  float64 
 7   Regionname  20170 non-null  category
dtypes: category(3), float64(4), int64(1)
memory usage: 848.9 KB


In [11]:
melb_cleaned.describe()

,Rooms,Price,Distance,Bathroom,Car
count,20170.000000,2.017000e+04,20170.000000,20170.000000,20170.000000
mean,3.041448,1.066963e+06,11.443009,1.563510,1.703272
std,0.914278,6.046802e+05,6.870251,0.632673,0.985066
min,1.000000,8.500000e+04,0.000000,0.000000,0.000000
25%,2.000000,6.520000e+05,6.600000,1.000000,1.000000
50%,3.000000,9.000000e+05,10.500000,1.000000,2.000000
75%,4.000000,1.320000e+06,14.200000,2.000000,2.000000
max,7.000000,6.000000e+06,48.100000,3.000000,18.000000


In [12]:
melb_cleaned.describe(exclude = 'number')

,Suburb,Type,Regionname
count,20170,20170,20170
unique,39,3,8
top,others,h,Southern Metropolitan
freq,12005,15121,6076


In [16]:
X, y = melb_cleaned.drop(columns = ['Price']), melb_cleaned['Price']

In [26]:
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error
from lightgbm import LGBMRegressor

In [15]:
model = LGBMRegressor()

In [19]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=13)
n_scores = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv = cv, n_jobs=-1)

print(np.mean(n_scores))

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


-303940.3627174365


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = 13, shuffle=True)

In [22]:
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000523 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 270
[LightGBM] [Info] Number of data points in the train set: 18153, number of used features: 7
[LightGBM] [Info] Start training from score 1065340.825538


LGBMRegressor()

In [23]:
y_predict = model.predict(X_test)

In [24]:
mean_absolute_error(y_predict, y_test)

193750.3277461643

In [25]:
mean_squared_error(y_predict, y_test)

98595616480.79807

In [28]:
root_mean_squared_error(y_predict, y_test)

313999.38930003997